In [ ]:
import h5py
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import copy
import pandas as pd
plt.style.use('ggplot')
# plt.rcParams["figure.figsize"] = (12,10)
plt.rcParams["figure.figsize"] = (10,8)

# Code to read Horizons.h5

In [ ]:
def make_Bh_pandas(h5_dir):
    # Empty dataframe
    df = pd.DataFrame()
    
    # List of all the vars in the h5 file
    var_list = []
    h5_dir.visit(var_list.append)
    
    
    for var in var_list:
        # This means there is no time column
        # print(f"{var} : {h5_dir[var].shape}")
        if df.shape == (0,0):
            # data[:,0] is time and then we have the data
            data = h5_dir[var]
            
            # vars[:-4] to remove the .dat at the end
            col_names = make_col_names(var[:-4],data.shape[1]-1)
            col_names.append('t')
            # Reverse the list so that we get ["t","var_name"]
            col_names.reverse()            
            append_to_df(data[:],col_names,df)
            
        else:
            data = h5_dir[var]
            col_names = make_col_names(var[:-4],data.shape[1]-1)         
            append_to_df(data[:,1:],col_names,df)
            
    return df

def append_to_df(data,col_names,df):
    for i,col_name in enumerate(col_names):
        df[col_name] = data[:,i]
        
def make_col_names(val_name:str,val_size:int):
    col_names = []
    if val_size == 1:
        col_names.append(val_name)
    else:
        for i in range(val_size):
            col_names.append(val_name+f"_{i}")
    return col_names


def horizon_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            df_dict[key[:-4]] = make_Bh_pandas(hf[key])

    return df_dict

def read_horizon_across_Levs(path_list:list[Path]):
    df_listAB = []
    df_listC = []
    final_dict = {}
    for path in path_list:
        df_lev = horizon_to_pandas(path)
        # Either [AhA,AhB] or [AhA,AhB,AhC]
        if len(df_lev.keys()) > 1:
            df_listAB.append(df_lev)
        # Either [AhC] or [AhA,AhB,AhC]
        if (len(df_lev.keys()) == 1) or (len(df_lev.keys()) ==3):
            df_listC.append(df_lev)
    if len(df_listAB)==1:
        # There was only one lev
        final_dict = df_listAB[0]
    else:
        final_dict["AhA"] = pd.concat([df["AhA"] for df in df_listAB])
        final_dict["AhB"] = pd.concat([df["AhB"] for df in df_listAB])
        if len(df_listC) > 0:
            final_dict["AhC"] = pd.concat([df["AhC"] for df in df_listC])       
    
    return final_dict

In [ ]:
base_path = Path("/panfs/ds09/sxs/himanshu/gauge_stuff/gauge_driver_runs/runs/80_ngd_master_mr1_100/Ev/")
file_pattern = "Lev3_A?/Run/ApparentHorizons/Horizons.h5"
path_list = list(base_path.glob(file_pattern))
path_list

In [ ]:
df = read_horizon_across_Levs(path_list)
print(df.keys())
df["AhA"].keys()

In [ ]:
x = 't'
y = "chiMagInertial"
plt.plot(df['AhA'][x],df['AhA'][y],label="AhA")
plt.plot(df['AhB'][x],df['AhB'][y],label="AhB")
# plt.plot(df['AhC'][x],df['AhC'][y],label="AhC")
plt.xlabel(x)
plt.ylabel(y)
plt.legend()

In [ ]:
plt.plot(df['AhC'][x],df['AhC'][y],label="AhC")
plt.xlabel(x)
plt.ylabel(y)
plt.legend()